In [2]:
import pandas as pd
df = pd.read_excel("dataset.xlsx")

In [3]:
df=df[["teaching","teaching.1"]]
column_rename={"teaching":"output","teaching.1":"message"}
df.rename(columns=column_rename,inplace=True)

In [4]:
df

,output,message
0,0,teacher are punctual but they should also give...
1,1,Good
2,1,Excellent lectures are delivered by teachers a...
3,1,Good
4,1,teachers give us all the information required ...
...,...,...
180,1,intraction is good and leacture delivery also ...
181,1,all the given terms are good regarding the uni...
182,1,All the terms are good regarding the universit...
183,-1,Some of the teacher are un experienced. Also t...


In [6]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [7]:
import re
from nltk.corpus import stopwords
corpus = []
for i in range(len(df)):
    review = re.sub("^a-zA-Z"," ",df["message"][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

In [14]:
df = pd.DataFrame({"message":corpus,"output":df["output"]})

In [30]:
df = df[["Summary","Sentiment"]]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.metrics import accuracy_score,classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

import re
from nltk.corpus import stopwords
corpus = []
for i in range(len(df)):
    review = re.sub("^a-zA-Z"," ",df["message"][i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

early_stopping = EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)

X = df["message"]
y = df["output"]


vectorizer = TfidfVectorizer(ngram_range=(1,3))
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

y_train_shifted = to_categorical(y_train,num_classes=3)
y_test_shifted = to_categorical(y_test,num_classes=3)

X_train_transformed = vectorizer.fit_transform(X_train).toarray()
X_test_transformed = vectorizer.transform(X_test).toarray()

model = Sequential([
    Dense(128,activation="relu",kernel_regularizer=l2(0.001),input_shape=(X_train_transformed.shape[1],)),
    Dropout(0.5),
    Dense(64,activation="relu",kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(3,activation="softmax"),
])
model.compile(optimizer=Adam(learning_rate=0.01),loss="categorical_crossentropy",metrics=["accuracy"])
history=model.fit(X_train_transformed,y_train_shifted,epochs = 20,batch_size=32,validation_data=(X_test_transformed,y_test_shifted),callbacks=[early_stopping])

Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.5608 - loss: 1.2260 - val_accuracy: 0.7568 - val_loss: 0.7940
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.7195 - loss: 0.7426 - val_accuracy: 0.7568 - val_loss: 0.7583
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.7186 - loss: 0.6451 - val_accuracy: 0.7568 - val_loss: 0.6972
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.7757 - loss: 0.5666 - val_accuracy: 0.7838 - val_loss: 0.6937
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8883 - loss: 0.4910 - val_accuracy: 0.7838 - val_loss: 0.7588
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.8902 - loss: 0.5280 - val_accuracy: 0.7297 - val_loss: 0.7524
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9291 - loss: 0.3922 - val_accuracy: 0.7027 - val_loss: 0.7288
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9184 - loss: 0.3722 - val_accuracy: 0.7297 - val_loss: 0.7811


## New Dataset

In [65]:
df = pd.read_csv("data/Dataset-SA.csv")

In [62]:
from sklearn.preprocessing import LabelEncoder
lbl_encoder = LabelEncoder()

In [66]:
X = df["Summary"]
y = df["Sentiment"]

In [59]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [60]:
y_train_encoded = lbl_encoder.fit_transform(y_train)
y_test_encoded = lbl_encoder.transform(y_test)

In [68]:
import re
from nltk.corpus import stopwords
from tqdm import tqdm
corpus = []
X = X.astype(str).fillna("")
for i in tqdm(range(len(df))):
    review = re.sub("[^a-zA-Z]"," ",X.iloc[i])
    review = review.lower()
    review = review.split()
    review = [lemmatizer.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

  0%|          | 0/205052 [00:00<?, ?it/s]

100%|██████████| 205052/205052 [04:58<00:00, 687.29it/s] 


In [71]:
df = pd.DataFrame({"message":corpus,"output":y})

In [72]:
df.to_csv("data/lematized_dataset.csv",index=False)

In [73]:
df

,message,output
0,great cooler excellent air flow price amazing ...,positive
1,best budget fit cooler nice cooling,positive
2,quality good power air decent,positive
3,bad product fan,negative
4,ok ok product,neutral
...,...,...
205047,good product,positive
205048,nice,positive
205049,nice fast delivery,positive
205050,awesome product,positive


## Training the Model

In [1]:
import pandas as pd
df = pd.read_csv("data/lematized_dataset.csv")

In [2]:
df["output"].value_counts()

output
positive    166581
negative     28232
neutral      10239
Name: count, dtype: int64

In [3]:
X = df["message"]
X = X.astype(str).fillna("")
y = df["output"]

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2),max_features=50000)
X_train_transformed = vectorizer.fit_transform(X_train)
X_test_transformed = vectorizer.transform(X_test)

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train_encoded_lbl = encoder.fit_transform(y_train)
y_test_encoded_lbl = encoder.transform(y_test)

In [7]:
from tensorflow.keras.utils import to_categorical
y_train_encoded_onehot = to_categorical(y_train_encoded_lbl)
y_test_encoded_onehot = to_categorical(y_test_encoded_lbl)

Training the ANN Model

In [73]:
X_train_transformed

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1078770 stored elements and shape (164041, 50000)>

In [74]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor="val_loss",patience=3,restore_best_weights=True)
model = Sequential([
    Dense(128,activation="relu",input_shape=(X_train_transformed.shape[1],)),
    Dropout(0.25),
    Dense(64,activation="relu"),
    Dropout(0.25),
    Dense(3,activation="softmax")
])

c:\Users\HP\OneDrive\Documents\FeedbackPrediction\feedpred\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [75]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [76]:
history = model.fit(X_train_transformed,y_train_encoded_onehot,validation_data=(X_test_transformed,y_test_encoded_onehot),epochs=20,batch_size=128,callbacks=[early_stopping])

Epoch 1/20
1282/1282 ━━━━━━━━━━━━━━━━━━━━ 144s 111ms/step - accuracy: 0.8849 - loss: 0.3701 - val_accuracy: 0.9217 - val_loss: 0.2362
Epoch 2/20
1282/1282 ━━━━━━━━━━━━━━━━━━━━ 147s 114ms/step - accuracy: 0.9386 - loss: 0.1881 - val_accuracy: 0.9201 - val_loss: 0.2460
Epoch 3/20
1282/1282 ━━━━━━━━━━━━━━━━━━━━ 143s 111ms/step - accuracy: 0.9510 - loss: 0.1525 - val_accuracy: 0.9175 - val_loss: 0.2733
Epoch 4/20
1282/1282 ━━━━━━━━━━━━━━━━━━━━ 140s 109ms/step - accuracy: 0.9576 - loss: 0.1324 - val_accuracy: 0.9177 - val_loss: 0.3022


In [78]:
from tensorflow.keras.models import save_model
model.save("models/final.h5")

In [79]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test_transformed)
print(classification_report(y_test_encoded_lbl, y_pred.argmax(axis=1)))


1282/1282 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step
              precision    recall  f1-score   support

           0       0.81      0.76      0.78      5557
           1       0.75      0.46      0.57      2033
           2       0.95      0.98      0.96     33421

    accuracy                           0.92     41011
   macro avg       0.83      0.73      0.77     41011
weighted avg       0.92      0.92      0.92     41011



In [8]:
import pickle
with open("models/vectorizer.pkl", "wb") as file:
    pickle.dump(vectorizer,file)
    